In [1]:
import pandas as pd
import numpy as np

#### train 데이터 필지 뽑아서 행정동&필지번호 합치기

In [2]:
raw_data = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/raw/필지단위 소상공인 매출등급 정보.csv', encoding='cp949')
match = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/필지번호_행정동.csv', encoding='cp949')
test = pd.read_excel('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/raw/소상공인 매출등급 예상 대상 필지.xlsx')

In [3]:
# train이면 1, test면 0 변수 생성
raw_data['train'] = 1

In [4]:
# test 데이터는 202301과 202302가 열별로 되어있어 각각 데이터프레임 만든 후 raw데이터 밑으로 붙이기
test_2301 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302
test_2301['train'] = 0
test_2302['train'] = 0

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\1090221059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\1090221059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [5]:
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'train']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'train']]

In [6]:
raw_data = pd.concat([raw_data,test_2301, test_2302])

In [7]:
print(raw_data)
print(match.head())

      DATA_CRTR_YM            LT_UNQ_NO  STDG_EMD_CD INDUSTRY_CD  SLS_GRD  \
0           202201  1111010100100030100     11110101         A03      4.0   
1           202201  1111010100100480000     11110101         A01      2.0   
2           202201  1111010100100590001     11110101         A11      3.0   
3           202201  1111010100100590005     11110101         A03      2.0   
4           202201  1111010100100690000     11110101         A01      2.0   
...            ...                  ...          ...         ...      ...   
9995        202302  1130510100100830004     11305101         A03      NaN   
9996        202302  1135010300107450000     11350103         A13      NaN   
9997        202302  1150010300103400017     11500103         A06      NaN   
9998        202302  1144012700116070000     11440127         A03      NaN   
9999        202302  1126010400100770063     11260104         A03      NaN   

      train  
0         1  
1         1  
2         1  
3         1  
4    

In [8]:
# 신사동이 관악구, 강남구 둘 다에 있어서 처리해줘야함. 이외에 구는 경계에 걸친 애들
for i in range(len(match)):
    if match['ADM_NM'][i] == '신사동':
        if (match['SGG_NM'][i] == '관악구') or (match['SGG_NM'][i] == '동작구'):
            match['ADM_NM'][i] = '신사동1'
        elif (match['SGG_NM'][i] == '강남구') or (match['SGG_NM'][i] == '서초구') or (match['SGG_NM'][i] == '용산구')or (match['SGG_NM'][i] == '성동구'):
            match['ADM_NM'][i] = '신사동2'

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\568015034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['ADM_NM'][i] = '신사동2'
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\568015034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['ADM_NM'][i] = '신사동1'


##### raw데이터에서 매칭 안되는 필지번호는 분리

In [9]:
X = pd.DataFrame(raw_data['LT_UNQ_NO'].unique())
Y = pd.DataFrame(match['PNU'].unique())
PNU_NOTIN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique())-set(match['PNU'].unique()))
PNU_IN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique()) & set(match['PNU'].unique()))
PNU_NOTIN_data.rename(columns={0:'PNU'}, inplace=True)
PNU_IN_data.rename(columns={0:'PNU'}, inplace=True)

In [10]:
print(len(X))
print(len(PNU_IN_data) + len(PNU_NOTIN_data))
print(len(PNU_IN_data))
print(len(PNU_NOTIN_data))

74303
74303
74199
104


In [11]:
raw_in_match = raw_data[raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]
raw_notin_match = raw_data[~raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]

In [12]:
print(len(raw_data))
print(len(raw_in_match) + len(raw_notin_match))
print(len(raw_in_match))
print(len(raw_notin_match))

1176040
1176040
1172902
3138


In [13]:
# 데이터에 있는 필지번호와 match를 merge
df = pd.merge(PNU_IN_data, match, how='left')

# 중복 필지 중 하나만 남기고 제거
df.drop_duplicates(subset='PNU', keep='first', inplace=True, ignore_index=False)

In [14]:
print(len(PNU_IN_data))
print(len(df))

74199
74199


In [15]:
# raw데이터에 행정동명 합치기
raw_in_match = pd.merge(raw_in_match, df[['PNU', 'ADM_CD', 'ADM_NM']], how='left', left_on = 'LT_UNQ_NO', right_on = 'PNU')

In [16]:
raw_in_match = raw_in_match[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD', 'train']]

In [17]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 1172901
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 80.5+ MB


#### 점포수 데이터 raw에 붙이기

In [18]:
# raw_in_match에서 '상일1동', '상일2동'이 데이터에서는 '상일동'. 따라서 raw_in_match의 '상일1동', '상일2동'을 '상일동'으로 바꾸어줘야함
raw_in_match['ADM_NM'][raw_in_match['ADM_NM'] == '상일1동'] = '상일동'
raw_in_match['ADM_NM'][raw_in_match['ADM_NM'] == '상일2동'] = '상일동'

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\1287381924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_in_match['ADM_NM'][raw_in_match['ADM_NM'] == '상일2동'] = '상일동'


In [19]:
num_store = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_점포수_1.csv', encoding = 'cp949')
num_store.head()

,행정구역,생활밀접업종,22-1전체점포수,22-1프랜차이즈점포수,22-1일반점포수,22-2전체점포수,22-2프랜차이즈점포수,22-2일반점포수,22-3전체점포수,22-3프랜차이즈점포수,22-3일반점포수,22-4전체점포수,22-4프랜차이즈점포수,22-4일반점포수,23-1전체점포수,23-1프랜차이즈점포수,23-1일반점포수
0,청운효자동,전체,296,5,291,298,5,293,305,6,299,308,6,302,312,6,306
1,사직동,전체,991,121,870,996,118,878,994,120,874,"1,001",121,880,995,117,878
2,삼청동,전체,257,28,229,259,29,230,259,30,229,266,30,236,270,29,241
3,부암동,전체,132,15,117,137,15,122,135,14,121,136,14,122,133,13,120
4,평창동,전체,144,15,129,144,15,129,139,15,124,141,15,126,143,16,127


In [20]:
# 업종 코드와 업종명을 매핑하는 딕셔너리 생성
industry_mapping = {
    'A01': '한식음식점',
    'A02': '치킨전문점',
    'A03': '전체',
    'A10': '전체',
    'A04': '제과점',
    'A05': '패스트푸드점',
    'A07': '패스트푸드점',
    'A06': '분식전문점',
    'A08': '커피-음료',
    'A09': '커피-음료',
    'A15': '커피-음료',
    'A11': '중식음식점',
    'A12': '호프-간이주점',
    'A13': '일식음식점',
    'A14': '양식음식점'
}

# '생활밀접업종' 열 추가 및 매핑
raw_in_match['생활밀접업종'] = raw_in_match['INDUSTRY_CD'].map(industry_mapping)

print(raw_in_match)

         DATA_CRTR_YM            LT_UNQ_NO  STDG_EMD_CD    ADM_CD ADM_NM  \
0              202201  1111010100100030100     11110101  11010720  청운효자동   
1              202201  1111010100100480000     11110101  11010720  청운효자동   
2              202201  1111010100100590001     11110101  11010720  청운효자동   
3              202201  1111010100100590005     11110101  11010720  청운효자동   
4              202201  1111010100100690000     11110101  11010720  청운효자동   
...               ...                  ...          ...       ...    ...   
1172897        202302  1130510100100830004     11305101  11090710    송중동   
1172898        202302  1135010300107450000     11350103  11110560   공릉2동   
1172899        202302  1150010300103400017     11500103  11160630   화곡8동   
1172900        202302  1144012700116070000     11440127  11140740    상암동   
1172901        202302  1126010400100770063     11260104  11070640    묵1동   

        INDUSTRY_CD  SLS_GRD  train 생활밀접업종  
0               A03      4.0      1     전체

In [21]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], num_store[['행정구역', '생활밀접업종', '22-1전체점포수', '22-1프랜차이즈점포수', '22-1일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], num_store[['행정구역', '생활밀접업종', '22-2전체점포수', '22-2프랜차이즈점포수', '22-2일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], num_store[['행정구역', '생활밀접업종', '22-3전체점포수', '22-3프랜차이즈점포수', '22-3일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], num_store[['행정구역', '생활밀접업종', '22-4전체점포수', '22-4프랜차이즈점포수', '22-4일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], num_store[['행정구역', '생활밀접업종', '23-1전체점포수', '23-1프랜차이즈점포수', '23-1일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')

In [22]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[10] = '전체점포수'
col_lst[11] = '프랜차이즈점포수'
col_lst[12] = '일반점포수'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[10] = '전체점포수'
col_lst[11] = '프랜차이즈점포수'
col_lst[12] = '일반점포수'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[10] = '전체점포수'
col_lst[11] = '프랜차이즈점포수'
col_lst[12] = '일반점포수'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[10] = '전체점포수'
col_lst[11] = '프랜차이즈점포수'
col_lst[12] = '일반점포수'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[10] = '전체점포수'
col_lst[11] = '프랜차이즈점포수'
col_lst[12] = '일반점포수'
raw_23_1.columns = col_lst

In [23]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [24]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
 8   생활밀접업종        1172902 non-null  object 
 9   전체점포수         1172902 non-null  object 
 10  프랜차이즈점포수      1172902 non-null  int64  
 11  일반점포수         1172902 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 116.3+ MB


##### A03, A10 비율 곱하기

In [25]:
# A03, A10의 비율을 계산합니다.
len(raw_data[raw_data['INDUSTRY_CD']=='A03'])/len(raw_data) # 0.2918
len(raw_data[raw_data['INDUSTRY_CD']=='A10'])/len(raw_data) #0.0044

0.0044012108431686

In [26]:
# '전체점포수'와 '일반점포수' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
raw_in_match['전체점포수'] = raw_in_match['전체점포수'].str.replace(',', '').astype(int)
raw_in_match['일반점포수'] = raw_in_match['일반점포수'].str.replace(',', '').astype(int)

# 비율을 적용할 조건을 설정합니다.
condition_A03 = (raw_in_match['INDUSTRY_CD'] == 'A03')
condition_A10 = (raw_in_match['INDUSTRY_CD'] == 'A10')

# 데이터프레임에 조건을 적용하여 비율을 계산합니다.
raw_in_match.loc[condition_A03, ['전체점포수', '프랜차이즈점포수', '일반점포수']] *= 0.29
raw_in_match.loc[condition_A10, ['전체점포수', '프랜차이즈점포수', '일반점포수']] *= 0.0044

In [27]:
raw_in_match.isna().sum()

DATA_CRTR_YM        0
LT_UNQ_NO           0
STDG_EMD_CD         0
ADM_CD              0
ADM_NM              0
INDUSTRY_CD         0
SLS_GRD         19970
train               0
생활밀접업종              0
전체점포수               0
프랜차이즈점포수            0
일반점포수               0
dtype: int64

#### 인구수 데이터 합치기

In [28]:
num_pop = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_인구수_1.csv', encoding='cp949')
num_pop.head()

,행정구역,22-1길단위유동인구,22-1주거인구,22-1직장인구,22-2길단위유동인구,22-2주거인구,22-2직장인구,22-3길단위유동인구,22-3주거인구,22-3직장인구,22-4길단위유동인구,22-4주거인구,22-4직장인구,23-1길단위유동인구,23-1주거인구,23-1직장인구
0,청운효자동,"14,929",51,44,"16,209",51,44,"14,843",51,44,"15,269",51,44,"14,633",54,44
1,사직동,"29,283",82,412,"34,630",82,412,"33,738",82,412,"34,815",82,412,"33,445",73,412
2,삼청동,"5,905",19,42,"6,885",19,42,"6,302",19,42,"6,649",19,42,"6,696",19,42
3,부암동,"5,778",45,19,"6,011",45,19,"5,468",45,19,"5,248",45,19,"4,970",44,19
4,평창동,987,22,28,986,22,28,975,22,28,"1,019",22,28,989,21,28


In [29]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], num_pop[['행정구역', '22-1길단위유동인구', '22-1주거인구', '22-1직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], num_pop[['행정구역', '22-2길단위유동인구', '22-2주거인구', '22-2직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], num_pop[['행정구역', '22-3길단위유동인구', '22-3주거인구', '22-3직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], num_pop[['행정구역', '22-4길단위유동인구', '22-4주거인구', '22-4직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], num_pop[['행정구역', '23-1길단위유동인구', '23-1주거인구', '23-1직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')

In [30]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[13] = '길단위유동인구'
col_lst[14] = '주거인구'
col_lst[15] = '직장인구'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[13] = '길단위유동인구'
col_lst[14] = '주거인구'
col_lst[15] = '직장인구'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[13] = '길단위유동인구'
col_lst[14] = '주거인구'
col_lst[15] = '직장인구'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[13] = '길단위유동인구'
col_lst[14] = '주거인구'
col_lst[15] = '직장인구'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[13] = '길단위유동인구'
col_lst[14] = '주거인구'
col_lst[15] = '직장인구'
raw_23_1.columns = col_lst

In [31]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [32]:
# '길단위유동인구'와 '직장인구' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
raw_in_match['길단위유동인구'] = raw_in_match['길단위유동인구'].str.replace(',', '').astype(int)
raw_in_match['직장인구'] = raw_in_match['직장인구'].str.replace(',', '').astype(int)

In [33]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 15 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
 8   생활밀접업종        1172902 non-null  object 
 9   전체점포수         1172902 non-null  float64
 10  프랜차이즈점포수      1172902 non-null  float64
 11  일반점포수         1172902 non-null  float64
 12  길단위유동인구       1172902 non-null  int32  
 13  주거인구          1172902 non-null  int64  
 14  직장인구          1172902 non-null  int32  
dtypes: float64(4), int32(2), int64(6), object(3)
memory usage: 134.2+ MB


#### 개폐업수 합치기

In [34]:
open_close = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_개폐업수_1.csv', encoding='cp949')
open_close.head()

,행정구역,생활밀접업종,22-1개업수,22-1폐업수,22-1개업률,22-1폐업률,22-2개업수,22-2폐업수,22-2개업률,22-2폐업률,...,22-3개업률,22-3폐업률,22-4개업수,22-4폐업수,22-4개업률,22-4폐업률,23-1개업수,23-1폐업수,23-1개업률,23-1폐업률
0,청운효자동,제과점,3,0,13.0,0.0,0,0,0.0,0.0,...,0.0,9.5,0,2,0.0,10.5,1,0,5.0,0.0
1,사직동,제과점,1,0,3.1,0.0,0,0,0.0,0.0,...,3.1,3.1,0,0,0.0,0.0,0,0,0.0,0.0
2,삼청동,제과점,0,0,0.0,0.0,0,1,0.0,7.1,...,7.7,15.4,2,0,13.3,0.0,0,0,0.0,0.0
3,부암동,제과점,1,0,14.3,0.0,0,0,0.0,0.0,...,12.5,0.0,0,0,0.0,0.0,0,1,0.0,14.3
4,평창동,제과점,0,0,0.0,0.0,0,0,0.0,0.0,...,0.0,0.0,0,1,0.0,20.0,0,0,0.0,0.0


In [35]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], open_close[['행정구역', '생활밀접업종', '22-1개업수', '22-1폐업수', '22-1개업률', '22-1폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], open_close[['행정구역', '생활밀접업종', '22-2개업수', '22-2폐업수', '22-2개업률', '22-2폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], open_close[['행정구역', '생활밀접업종', '22-3개업수', '22-3폐업수', '22-3개업률', '22-3폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], open_close[['행정구역', '생활밀접업종', '22-4개업수', '22-4폐업수', '22-4개업률', '22-4폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], open_close[['행정구역', '생활밀접업종', '23-1개업수', '23-1폐업수', '23-1개업률', '23-1폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')

In [36]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[16] = '개업수'
col_lst[17] = '폐업수'
col_lst[18] = '개업률'
col_lst[19] = '폐업률'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[16] = '개업수'
col_lst[17] = '폐업수'
col_lst[18] = '개업률'
col_lst[19] = '폐업률'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[16] = '개업수'
col_lst[17] = '폐업수'
col_lst[18] = '개업률'
col_lst[19] = '폐업률'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[16] = '개업수'
col_lst[17] = '폐업수'
col_lst[18] = '개업률'
col_lst[19] = '폐업률'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[16] = '개업수'
col_lst[17] = '폐업수'
col_lst[18] = '개업률'
col_lst[19] = '폐업률'
raw_23_1.columns = col_lst

In [37]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [38]:
# 비율을 적용할 조건을 설정합니다.
condition_A03 = (raw_in_match['INDUSTRY_CD'] == 'A03')
condition_A10 = (raw_in_match['INDUSTRY_CD'] == 'A10')

# 데이터프레임에 조건을 적용하여 비율을 계산합니다.
raw_in_match.loc[condition_A03, ['개업수', '폐업수', '개업률', '폐업률']] *= 0.29
raw_in_match.loc[condition_A10, ['개업수', '폐업수', '개업률', '폐업률']] *= 0.0044

In [39]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 19 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
 8   생활밀접업종        1172902 non-null  object 
 9   전체점포수         1172902 non-null  float64
 10  프랜차이즈점포수      1172902 non-null  float64
 11  일반점포수         1172902 non-null  float64
 12  길단위유동인구       1172902 non-null  int32  
 13  주거인구          1172902 non-null  int64  
 14  직장인구          1172902 non-null  int32  
 15  개업수           1172902 non-null  float64
 16  폐업수           1172902 non-null  float64
 17  개업률           1172902 non-nul

#### 임대료 합치기

In [40]:
fee_rent = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_임대료_희중(결측값).csv')
fee_rent.head()

,행정구역,22-1 전체,22-1 1층,22-1 1층 외,22-2 전체,22-2 1층,22-2 1층 외,22-3 전체,22-3 1층,22-3 1층 외,22-4 전체,22-4 1층,22-4 1층 외,23-1 전체,23-1 1층,23-1 1층 외
0,청운효자동,129755,181103,78407,147878,212258,83498,142725,195183,90268,155797,224849,86744,151256,225252,77260
1,사직동,145664,185390,105938,162108,217048,107167,166024,228857,103192,173553,246659,100447,150227,209365,91088
2,삼청동,175042,185890,164194,161693,180283,143103,157572,194483,120661,168139,200614,135664,190228,226225,154231
3,부암동,88340,94506,82173,74155,93432,54879,93334,118701,67967,99441,122603,76278,107082,107082,82139
4,평창동,92288,115285,69292,110528,148983,72074,111164,123164,99164,121969,132186,111752,116535,126296,116535


In [41]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], fee_rent[['행정구역', '22-1 전체', '22-1 1층', '22-1 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], fee_rent[['행정구역', '22-2 전체', '22-2 1층', '22-2 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], fee_rent[['행정구역', '22-3 전체', '22-3 1층', '22-3 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], fee_rent[['행정구역', '22-4 전체', '22-4 1층', '22-4 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], fee_rent[['행정구역', '23-1 전체', '23-1 1층', '23-1 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')

In [42]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[20] = '전체 임대료'
col_lst[21] = '1층 임대료'
col_lst[22] = '1층 외 임대료'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[20] = '전체 임대료'
col_lst[21] = '1층 임대료'
col_lst[22] = '1층 외 임대료'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[20] = '전체 임대료'
col_lst[21] = '1층 임대료'
col_lst[22] = '1층 외 임대료'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[20] = '전체 임대료'
col_lst[21] = '1층 임대료'
col_lst[22] = '1층 외 임대료'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[20] = '전체 임대료'
col_lst[21] = '1층 임대료'
col_lst[22] = '1층 외 임대료'
raw_23_1.columns = col_lst

In [43]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [44]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 22 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   train         1172902 non-null  int64  
 8   생활밀접업종        1172902 non-null  object 
 9   전체점포수         1172902 non-null  float64
 10  프랜차이즈점포수      1172902 non-null  float64
 11  일반점포수         1172902 non-null  float64
 12  길단위유동인구       1172902 non-null  int32  
 13  주거인구          1172902 non-null  int64  
 14  직장인구          1172902 non-null  int32  
 15  개업수           1172902 non-null  float64
 16  폐업수           1172902 non-null  float64
 17  개업률           1172902 non-nul

In [45]:
raw_in_match.isna().sum()

DATA_CRTR_YM        0
LT_UNQ_NO           0
STDG_EMD_CD         0
ADM_CD              0
ADM_NM              0
INDUSTRY_CD         0
SLS_GRD         19970
train               0
생활밀접업종              0
전체점포수               0
프랜차이즈점포수            0
일반점포수               0
길단위유동인구             0
주거인구                0
직장인구                0
개업수                 0
폐업수                 0
개업률                 0
폐업률                 0
전체 임대료              0
1층 임대료              0
1층 외 임대료            0
dtype: int64

### raw_notin_match 합치기

In [46]:
raw_notin_match

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train
280,202201,1111011000102450006,11110110,A03,2.0,1
12662,202201,1120010700103770000,11200107,A01,3.0,1
12663,202201,1120010700103770000,11200107,A08,2.0,1
17432,202201,1121510500108610000,11215105,A08,4.0,1
24678,202201,1126010500106150005,11260105,A13,5.0,1
...,...,...,...,...,...,...
6356,202302,1165010100104500013,11650101,A01,NaN,0
7203,202302,1165010100110180001,11650101,A01,NaN,0
7378,202302,1138011400100250000,11380114,A06,NaN,0
7384,202302,1171010800106520000,11710108,A01,NaN,0


##### 공통되는 부분 필지번호 찾기

In [47]:
# 필지번호 10자리까지가 행정구역코드여서 필지번호를 문자열로 만든 다음 10번째 자리까지 끊기
pnu_match = match['PNU'].astype(str)
pnu_notin_match = raw_notin_match['LT_UNQ_NO'].astype(str)

def str_slicing(s):
    return s[:10]

pnu_match_slice = list(map(str_slicing, pnu_match))
pnu_notin_match_slice = list(map(str_slicing, pnu_notin_match))

In [48]:
set(pnu_notin_match_slice) - set(pnu_match_slice) # 10번째 자리까지 slice하면 match에 있는 필지번호(10자리까지)에 모두 포함됨

set()

##### raw_notin_match에 slicing한 필지번호 추가해서 matching

In [49]:
raw_notin_match['LT_UNQ_NO_slice'] = raw_notin_match['LT_UNQ_NO'].astype(str)
raw_notin_match['LT_UNQ_NO_slice'] = list(map(str_slicing, raw_notin_match['LT_UNQ_NO_slice']))
match['PNU_slice'] = match['PNU'].astype(str)
match['PNU_slice'] = list(map(str_slicing, match['PNU_slice']))

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\962864094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_notin_match['LT_UNQ_NO_slice'] = raw_notin_match['LT_UNQ_NO'].astype(str)
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\962864094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_notin_match['LT_UNQ_NO_slice'] = list(map(str_slicing, raw_notin_match['LT_UNQ_NO_slice']))


In [50]:
raw_notin_match

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,LT_UNQ_NO_slice
280,202201,1111011000102450006,11110110,A03,2.0,1,1111011000
12662,202201,1120010700103770000,11200107,A01,3.0,1,1120010700
12663,202201,1120010700103770000,11200107,A08,2.0,1,1120010700
17432,202201,1121510500108610000,11215105,A08,4.0,1,1121510500
24678,202201,1126010500106150005,11260105,A13,5.0,1,1126010500
...,...,...,...,...,...,...,...
6356,202302,1165010100104500013,11650101,A01,NaN,0,1165010100
7203,202302,1165010100110180001,11650101,A01,NaN,0,1165010100
7378,202302,1138011400100250000,11380114,A06,NaN,0,1138011400
7384,202302,1171010800106520000,11710108,A01,NaN,0,1171010800


In [51]:
raw_notin_match1 = raw_notin_match

In [52]:
match.drop(columns='PNU', inplace=True)
match.drop_duplicates(inplace=True)

In [53]:
match

,SGG_CD,SGG_NM,EMD_CD,EMD_NM,ADM_CD,ADM_NM,PNU_slice
0,11110,종로구,11110101,청운동,11010720,청운효자동,1111010100
6,11110,종로구,11110101,청운동,11010550,부암동,1111010100
792,11110,종로구,11110101,청운동,11010540,삼청동,1111010100
825,11110,종로구,11110102,신교동,11010720,청운효자동,1111010200
1220,11110,종로구,11110103,궁정동,11010720,청운효자동,1111010300
...,...,...,...,...,...,...,...
937427,11740,강동구,11740109,천호동,11240510,풍납1동,1174010900
938127,11740,강동구,11740109,천호동,11050630,광장동,1174010900
938483,11740,강동구,11740110,강일동,11250750,강일동,1174011000
939255,11740,강동구,11740110,강일동,11250560,고덕2동,1174011000


In [54]:
pd.merge(raw_notin_match, match[['PNU_slice', 'ADM_CD', 'ADM_NM']], left_on='LT_UNQ_NO_slice', right_on='PNU_slice', how = 'left')

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,LT_UNQ_NO_slice,PNU_slice,ADM_CD,ADM_NM
0,202201,1111011000102450006,11110110,A03,2.0,1,1111011000,1111011000,11010720,청운효자동
1,202201,1111011000102450006,11110110,A03,2.0,1,1111011000,1111011000,11010530,사직동
2,202201,1120010700103770000,11200107,A01,3.0,1,1120010700,1120010700,11040550,사근동
3,202201,1120010700103770000,11200107,A01,3.0,1,1120010700,1120010700,11040540,마장동
4,202201,1120010700103770000,11200107,A01,3.0,1,1120010700,1120010700,11040560,행당1동
...,...,...,...,...,...,...,...,...,...,...
28329,202302,1171010800106400007,11710108,A11,NaN,0,1171010800,1171010800,11240530,거여1동
28330,202302,1171010800106400007,11710108,A11,NaN,0,1171010800,1171010800,11240650,가락본동
28331,202302,1171010800106400007,11710108,A11,NaN,0,1171010800,1171010800,11240690,문정2동
28332,202302,1171010800106400007,11710108,A11,NaN,0,1171010800,1171010800,11240660,가락1동


In [55]:
raw_notin_match = pd.merge(raw_notin_match, match[['PNU_slice', 'ADM_CD', 'ADM_NM']], left_on='LT_UNQ_NO_slice', right_on='PNU_slice', how = 'left')
raw_notin_match.drop(columns=['PNU_slice'], axis=1, inplace=True)

In [56]:
raw_notin_match = raw_notin_match[['DATA_CRTR_YM', 'LT_UNQ_NO', 'LT_UNQ_NO_slice', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD',
       'train']]

In [57]:
raw_notin_match

,DATA_CRTR_YM,LT_UNQ_NO,LT_UNQ_NO_slice,STDG_EMD_CD,ADM_CD,ADM_NM,INDUSTRY_CD,SLS_GRD,train
0,202201,1111011000102450006,1111011000,11110110,11010720,청운효자동,A03,2.0,1
1,202201,1111011000102450006,1111011000,11110110,11010530,사직동,A03,2.0,1
2,202201,1120010700103770000,1120010700,11200107,11040550,사근동,A01,3.0,1
3,202201,1120010700103770000,1120010700,11200107,11040540,마장동,A01,3.0,1
4,202201,1120010700103770000,1120010700,11200107,11040560,행당1동,A01,3.0,1
...,...,...,...,...,...,...,...,...,...
28329,202302,1171010800106400007,1171010800,11710108,11240530,거여1동,A11,NaN,0
28330,202302,1171010800106400007,1171010800,11710108,11240650,가락본동,A11,NaN,0
28331,202302,1171010800106400007,1171010800,11710108,11240690,문정2동,A11,NaN,0
28332,202302,1171010800106400007,1171010800,11710108,11240660,가락1동,A11,NaN,0


##### 점포수 데이터 raw_notin에 붙이기

In [58]:
# raw_notin_match에서 '상일1동', '상일2동'이 데이터에서는 '상일동'. 따라서 raw_notin_match의 '상일1동', '상일2동'을 '상일동'으로 바꾸어줘야함
raw_notin_match['ADM_NM'][raw_notin_match['ADM_NM'] == '상일1동'] = '상일동'
raw_notin_match['ADM_NM'][raw_notin_match['ADM_NM'] == '상일2동'] = '상일동'

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_3640\117906907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_notin_match['ADM_NM'][raw_notin_match['ADM_NM'] == '상일2동'] = '상일동'


In [59]:
# 업종 코드와 업종명을 매핑하는 딕셔너리 생성
industry_mapping = {
    'A01': '한식음식점',
    'A02': '치킨전문점',
    'A03': '전체',
    'A10': '전체',
    'A04': '제과점',
    'A05': '패스트푸드점',
    'A07': '패스트푸드점',
    'A06': '분식전문점',
    'A08': '커피-음료',
    'A09': '커피-음료',
    'A15': '커피-음료',
    'A11': '중식음식점',
    'A12': '호프-간이주점',
    'A13': '일식음식점',
    'A14': '양식음식점'
}

# '생활밀접업종' 열 추가 및 매핑
raw_notin_match['생활밀접업종'] = raw_notin_match['INDUSTRY_CD'].map(industry_mapping)

print(raw_notin_match)

       DATA_CRTR_YM            LT_UNQ_NO LT_UNQ_NO_slice  STDG_EMD_CD  \
0            202201  1111011000102450006      1111011000     11110110   
1            202201  1111011000102450006      1111011000     11110110   
2            202201  1120010700103770000      1120010700     11200107   
3            202201  1120010700103770000      1120010700     11200107   
4            202201  1120010700103770000      1120010700     11200107   
...             ...                  ...             ...          ...   
28329        202302  1171010800106400007      1171010800     11710108   
28330        202302  1171010800106400007      1171010800     11710108   
28331        202302  1171010800106400007      1171010800     11710108   
28332        202302  1171010800106400007      1171010800     11710108   
28333        202302  1171010800106400007      1171010800     11710108   

         ADM_CD ADM_NM INDUSTRY_CD  SLS_GRD  train 생활밀접업종  
0      11010720  청운효자동         A03      2.0      1     전체  
1  

In [60]:
# 분기별로 merge
raw_notin_22_1 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] <= 202203], num_store[['행정구역', '생활밀접업종', '22-1전체점포수', '22-1프랜차이즈점포수', '22-1일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_22_2 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202206) & (raw_notin_match['DATA_CRTR_YM'] > 202203)], num_store[['행정구역', '생활밀접업종', '22-2전체점포수', '22-2프랜차이즈점포수', '22-2일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_22_3 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202209) & (raw_notin_match['DATA_CRTR_YM'] > 202206)], num_store[['행정구역', '생활밀접업종', '22-3전체점포수', '22-3프랜차이즈점포수', '22-3일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_22_4 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202212) & (raw_notin_match['DATA_CRTR_YM'] > 202209)], num_store[['행정구역', '생활밀접업종', '22-4전체점포수', '22-4프랜차이즈점포수', '22-4일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_23_1 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202302) & (raw_notin_match['DATA_CRTR_YM'] > 202212)], num_store[['행정구역', '생활밀접업종', '23-1전체점포수', '23-1프랜차이즈점포수', '23-1일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')

In [61]:
# 열 이름 통일
## 22-1
col_lst = list(raw_notin_22_1.columns)
col_lst[11] = '전체점포수'
col_lst[12] = '프랜차이즈점포수'
col_lst[13] = '일반점포수'
raw_notin_22_1.columns = col_lst
## 22-2
col_lst = list(raw_notin_22_2.columns)
col_lst[11] = '전체점포수'
col_lst[12] = '프랜차이즈점포수'
col_lst[13] = '일반점포수'
raw_notin_22_2.columns = col_lst
## 22-3
col_lst = list(raw_notin_22_3.columns)
col_lst[11] = '전체점포수'
col_lst[12] = '프랜차이즈점포수'
col_lst[13] = '일반점포수'
raw_notin_22_3.columns = col_lst
## 22-4
col_lst = list(raw_notin_22_4.columns)
col_lst[11] = '전체점포수'
col_lst[12] = '프랜차이즈점포수'
col_lst[13] = '일반점포수'
raw_notin_22_4.columns = col_lst
## 23-1
col_lst = list(raw_notin_23_1.columns)
col_lst[11] = '전체점포수'
col_lst[12] = '프랜차이즈점포수'
col_lst[13] = '일반점포수'
raw_notin_23_1.columns = col_lst

In [62]:
# 분기별 데이터 하나로 합치기
raw_notin_match = pd.concat([raw_notin_22_1, raw_notin_22_2, raw_notin_22_3, raw_notin_22_4, raw_notin_23_1])
raw_notin_match.drop(columns='행정구역', axis=1, inplace=True)

###### A03, A10 비율곱

In [63]:
# '전체점포수'와 '일반점포수' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
raw_notin_match['전체점포수'] = raw_notin_match['전체점포수'].str.replace(',', '').astype(int)
raw_notin_match['일반점포수'] = raw_notin_match['일반점포수'].str.replace(',', '').astype(int)

# 비율을 적용할 조건을 설정합니다.
condition_A03 = (raw_notin_match['INDUSTRY_CD'] == 'A03')
condition_A10 = (raw_notin_match['INDUSTRY_CD'] == 'A10')

# 데이터프레임에 조건을 적용하여 비율을 계산합니다.
raw_notin_match.loc[condition_A03, ['전체점포수', '프랜차이즈점포수', '일반점포수']] *= 0.29
raw_notin_match.loc[condition_A10, ['전체점포수', '프랜차이즈점포수', '일반점포수']] *= 0.0044

#### 인구수 데이터

In [64]:
# 분기별로 merge
raw_notin_22_1 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] <= 202203], num_pop[['행정구역', '22-1길단위유동인구', '22-1주거인구', '22-1직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_22_2 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202206) & (raw_notin_match['DATA_CRTR_YM'] > 202203)], num_pop[['행정구역', '22-2길단위유동인구', '22-2주거인구', '22-2직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_22_3 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202209) & (raw_notin_match['DATA_CRTR_YM'] > 202206)], num_pop[['행정구역', '22-3길단위유동인구', '22-3주거인구', '22-3직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_22_4 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202212) & (raw_notin_match['DATA_CRTR_YM'] > 202209)], num_pop[['행정구역', '22-4길단위유동인구', '22-4주거인구', '22-4직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_23_1 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202302) & (raw_notin_match['DATA_CRTR_YM'] > 202212)], num_pop[['행정구역', '23-1길단위유동인구', '23-1주거인구', '23-1직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')

In [65]:
# 열 이름 통일
## 22-1
col_lst = list(raw_notin_22_1.columns)
col_lst[14] = '길단위유동인구'
col_lst[15] = '주거인구'
col_lst[16] = '직장인구'
raw_notin_22_1.columns = col_lst
## 22-2
col_lst = list(raw_notin_22_2.columns)
col_lst[14] = '길단위유동인구'
col_lst[15] = '주거인구'
col_lst[16] = '직장인구'
raw_notin_22_2.columns = col_lst
## 22-3
col_lst = list(raw_notin_22_3.columns)
col_lst[14] = '길단위유동인구'
col_lst[15] = '주거인구'
col_lst[16] = '직장인구'
raw_notin_22_3.columns = col_lst
## 22-4
col_lst = list(raw_notin_22_4.columns)
col_lst[14] = '길단위유동인구'
col_lst[15] = '주거인구'
col_lst[16] = '직장인구'
raw_notin_22_4.columns = col_lst
## 23-1
col_lst = list(raw_notin_23_1.columns)
col_lst[14] = '길단위유동인구'
col_lst[15] = '주거인구'
col_lst[16] = '직장인구'
raw_notin_23_1.columns = col_lst

In [66]:
# 분기별 데이터 하나로 합치기
raw_notin_match = pd.concat([raw_notin_22_1, raw_notin_22_2, raw_notin_22_3, raw_notin_22_4, raw_notin_23_1])
raw_notin_match.drop(columns='행정구역', axis=1, inplace=True)

In [67]:
# '길단위유동인구'와 '직장인구' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
raw_notin_match['길단위유동인구'] = raw_notin_match['길단위유동인구'].str.replace(',', '').astype(int)
raw_notin_match['직장인구'] = raw_notin_match['직장인구'].str.replace(',', '').astype(int)

#### 개폐업수 데이터

In [68]:
# 분기별로 merge
raw_notin_22_1 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] <= 202203], open_close[['행정구역', '생활밀접업종', '22-1개업수', '22-1폐업수', '22-1개업률', '22-1폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_22_2 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202206) & (raw_notin_match['DATA_CRTR_YM'] > 202203)], open_close[['행정구역', '생활밀접업종', '22-2개업수', '22-2폐업수', '22-2개업률', '22-2폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_22_3 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202209) & (raw_notin_match['DATA_CRTR_YM'] > 202206)], open_close[['행정구역', '생활밀접업종', '22-3개업수', '22-3폐업수', '22-3개업률', '22-3폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_22_4 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202212) & (raw_notin_match['DATA_CRTR_YM'] > 202209)], open_close[['행정구역', '생활밀접업종', '22-4개업수', '22-4폐업수', '22-4개업률', '22-4폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_notin_23_1 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202302) & (raw_notin_match['DATA_CRTR_YM'] > 202212)], open_close[['행정구역', '생활밀접업종', '23-1개업수', '23-1폐업수', '23-1개업률', '23-1폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')

In [69]:
# 열 이름 통일
## 22-1
col_lst = list(raw_notin_22_1.columns)
col_lst[17] = '개업수'
col_lst[18] = '폐업수'
col_lst[19] = '개업률'
col_lst[20] = '폐업률'
raw_notin_22_1.columns = col_lst
## 22-2
col_lst = list(raw_notin_22_2.columns)
col_lst[17] = '개업수'
col_lst[18] = '폐업수'
col_lst[19] = '개업률'
col_lst[20] = '폐업률'
raw_notin_22_2.columns = col_lst
## 22-3
col_lst = list(raw_notin_22_3.columns)
col_lst[17] = '개업수'
col_lst[18] = '폐업수'
col_lst[19] = '개업률'
col_lst[20] = '폐업률'
raw_notin_22_3.columns = col_lst
## 22-4
col_lst = list(raw_notin_22_4.columns)
col_lst[17] = '개업수'
col_lst[18] = '폐업수'
col_lst[19] = '개업률'
col_lst[20] = '폐업률'
raw_notin_22_4.columns = col_lst
## 23-1
col_lst = list(raw_notin_23_1.columns)
col_lst[17] = '개업수'
col_lst[18] = '폐업수'
col_lst[19] = '개업률'
col_lst[20] = '폐업률'
raw_notin_23_1.columns = col_lst

In [70]:
# 분기별 데이터 하나로 합치기
raw_notin_match = pd.concat([raw_notin_22_1, raw_notin_22_2, raw_notin_22_3, raw_notin_22_4, raw_notin_23_1])
raw_notin_match.drop(columns='행정구역', axis=1, inplace=True)

In [71]:
# 비율을 적용할 조건을 설정합니다.
condition_A03 = (raw_notin_match['INDUSTRY_CD'] == 'A03')
condition_A10 = (raw_notin_match['INDUSTRY_CD'] == 'A10')

# 데이터프레임에 조건을 적용하여 비율을 계산합니다.
raw_notin_match.loc[condition_A03, ['개업수', '폐업수', '개업률', '폐업률']] *= 0.29
raw_notin_match.loc[condition_A10, ['개업수', '폐업수', '개업률', '폐업률']] *= 0.0044

#### 임대료 데이터

In [72]:
# 분기별로 merge
raw_notin_22_1 = pd.merge(raw_notin_match[raw_notin_match['DATA_CRTR_YM'] <= 202203], fee_rent[['행정구역', '22-1 전체', '22-1 1층', '22-1 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_22_2 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202206) & (raw_notin_match['DATA_CRTR_YM'] > 202203)], fee_rent[['행정구역', '22-2 전체', '22-2 1층', '22-2 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_22_3 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202209) & (raw_notin_match['DATA_CRTR_YM'] > 202206)], fee_rent[['행정구역', '22-3 전체', '22-3 1층', '22-3 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_22_4 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202212) & (raw_notin_match['DATA_CRTR_YM'] > 202209)], fee_rent[['행정구역', '22-4 전체', '22-4 1층', '22-4 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_notin_23_1 = pd.merge(raw_notin_match[(raw_notin_match['DATA_CRTR_YM'] <= 202302) & (raw_notin_match['DATA_CRTR_YM'] > 202212)], fee_rent[['행정구역', '23-1 전체', '23-1 1층', '23-1 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')

In [73]:
# 열 이름 통일
## 22-1
col_lst = list(raw_notin_22_1.columns)
col_lst[21] = '전체 임대료'
col_lst[22] = '1층 임대료'
col_lst[23] = '1층 외 임대료'
raw_notin_22_1.columns = col_lst
## 22-2
col_lst = list(raw_notin_22_2.columns)
col_lst[21] = '전체 임대료'
col_lst[22] = '1층 임대료'
col_lst[23] = '1층 외 임대료'
raw_notin_22_2.columns = col_lst
## 22-3
col_lst = list(raw_notin_22_3.columns)
col_lst[21] = '전체 임대료'
col_lst[22] = '1층 임대료'
col_lst[23] = '1층 외 임대료'
raw_notin_22_3.columns = col_lst
## 22-4
col_lst = list(raw_notin_22_4.columns)
col_lst[21] = '전체 임대료'
col_lst[22] = '1층 임대료'
col_lst[23] = '1층 외 임대료'
raw_notin_22_4.columns = col_lst
## 23-1
col_lst = list(raw_notin_23_1.columns)
col_lst[21] = '전체 임대료'
col_lst[22] = '1층 임대료'
col_lst[23] = '1층 외 임대료'
raw_notin_23_1.columns = col_lst

In [74]:
# 분기별 데이터 하나로 합치기
raw_notin_match = pd.concat([raw_notin_22_1, raw_notin_22_2, raw_notin_22_3, raw_notin_22_4, raw_notin_23_1])
raw_notin_match.drop(columns='행정구역', axis=1, inplace=True)

In [75]:
raw_notin_match.isna().sum()

DATA_CRTR_YM         0
LT_UNQ_NO            0
LT_UNQ_NO_slice      0
STDG_EMD_CD          0
ADM_CD               0
ADM_NM               0
INDUSTRY_CD          0
SLS_GRD            250
train                0
생활밀접업종               0
전체점포수                0
프랜차이즈점포수             0
일반점포수                0
길단위유동인구              0
주거인구                 0
직장인구                 0
개업수                  0
폐업수                  0
개업률                  0
폐업률                  0
전체 임대료               0
1층 임대료               0
1층 외 임대료             0
dtype: int64

### groupby mean

In [76]:
raw_notin_match.head()

,DATA_CRTR_YM,LT_UNQ_NO,LT_UNQ_NO_slice,STDG_EMD_CD,ADM_CD,ADM_NM,INDUSTRY_CD,SLS_GRD,train,생활밀접업종,...,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,전체 임대료,1층 임대료,1층 외 임대료
0,202201,1111011000102450006,1111011000,11110110,11010720,청운효자동,A03,2.0,1,전체,...,14929,51,44,3.48,2.03,1.189,0.696,129755,181103,78407
1,202201,1111011000102450006,1111011000,11110110,11010530,사직동,A03,2.0,1,전체,...,29283,82,412,6.09,5.51,0.609,0.551,145664,185390,105938
2,202201,1120010700103770000,1120010700,11200107,11040550,사근동,A01,3.0,1,한식음식점,...,34348,101,22,1.00,3.00,0.600,1.800,146898,178030,115766
3,202201,1120010700103770000,1120010700,11200107,11040540,마장동,A01,3.0,1,한식음식점,...,51686,210,71,3.00,5.00,2.400,4.000,112213,143574,80852
4,202201,1120010700103770000,1120010700,11200107,11040560,행당1동,A01,3.0,1,한식음식점,...,70185,255,43,2.00,6.00,1.500,4.500,119981,135400,104561


In [77]:
raw_notin_match.columns

Index(['DATA_CRTR_YM', 'LT_UNQ_NO', 'LT_UNQ_NO_slice', 'STDG_EMD_CD', 'ADM_CD',
       'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD', 'train', '생활밀접업종', '전체점포수',
       '프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률',
       '폐업률', '전체 임대료', '1층 임대료', '1층 외 임대료'],
      dtype='object')

In [78]:
# 행정코드 10자리, 업종코드, 연월로 그룹화하여 평균 구하기
grouped = raw_notin_match.groupby(['LT_UNQ_NO_slice', 'INDUSTRY_CD', 'DATA_CRTR_YM'])[['전체점포수',
       '프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률',
       '폐업률', '전체 임대료', '1층 임대료', '1층 외 임대료']].mean().reset_index()

In [79]:
grouped

,LT_UNQ_NO_slice,INDUSTRY_CD,DATA_CRTR_YM,전체점포수,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,전체 임대료,1층 임대료,1층 외 임대료
0,1111011000,A03,202201,186.615000,18.270000,168.345000,22106.000000,66.5,228.000000,4.785000,3.770000,0.899000,0.623500,137709.500000,183246.500000,92172.500000
1,1111011000,A03,202202,186.615000,18.270000,168.345000,22106.000000,66.5,228.000000,4.785000,3.770000,0.899000,0.623500,137709.500000,183246.500000,92172.500000
2,1111011000,A03,202203,186.615000,18.270000,168.345000,22106.000000,66.5,228.000000,4.785000,3.770000,0.899000,0.623500,137709.500000,183246.500000,92172.500000
3,1111011000,A03,202204,187.630000,17.835000,169.795000,25419.500000,66.5,228.000000,5.655000,4.640000,0.797500,0.638000,154993.000000,214653.000000,95332.500000
4,1111011000,A03,202205,187.630000,17.835000,169.795000,25419.500000,66.5,228.000000,5.655000,4.640000,0.797500,0.638000,154993.000000,214653.000000,95332.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,1174010300,A08,202208,29.166667,9.333333,19.833333,16795.000000,147.0,15.666667,1.833333,1.333333,5.933333,3.983333,156957.166667,212104.166667,101809.833333
1072,1174010300,A08,202209,29.166667,9.333333,19.833333,16795.000000,147.0,15.666667,1.833333,1.333333,5.933333,3.983333,156957.166667,212104.166667,101809.833333
1073,1174010300,A08,202210,30.500000,10.000000,20.500000,17015.166667,147.0,15.666667,2.666667,1.333333,8.183333,5.766667,152780.500000,204188.333333,101373.166667
1074,1174010300,A08,202211,30.500000,10.000000,20.500000,17015.166667,147.0,15.666667,2.666667,1.333333,8.183333,5.766667,152780.500000,204188.333333,101373.166667


In [80]:
raw_notin_match2 = pd.merge(raw_notin_match1, grouped, on=['LT_UNQ_NO_slice', 'INDUSTRY_CD', 'DATA_CRTR_YM'])
raw_notin_match2.drop(columns='LT_UNQ_NO_slice', inplace=True)

In [81]:
raw_notin_match2.head(5)

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,전체점포수,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,전체 임대료,1층 임대료,1층 외 임대료
0,202201,1111011000102450006,11110110,A03,2.0,1,186.615000,18.270000,168.345000,22106.000000,66.500000,228.000000,4.785000,3.770000,0.899000,0.623500,137709.500000,183246.500000,92172.500000
1,202201,1120010700103770000,11200107,A01,3.0,1,109.444444,10.888889,98.555556,74873.000000,299.111111,45.000000,2.000000,2.666667,1.877778,2.000000,125280.333333,146819.111111,103741.000000
2,202201,1120010700103770000,11200107,A08,2.0,1,44.888889,11.111111,33.777778,74873.000000,299.111111,45.000000,1.777778,1.333333,2.088889,3.233333,125280.333333,146819.111111,103741.000000
3,202201,1121510500108610000,11215105,A08,4.0,1,74.692308,20.230769,54.461538,59650.384615,198.153846,116.076923,2.692308,2.000000,4.369231,2.884615,148145.846154,178620.076923,117671.769231
4,202201,1126010500106150005,11260105,A13,5.0,1,8.200000,0.600000,7.600000,53457.800000,202.400000,16.600000,0.600000,0.400000,4.680000,6.000000,84054.600000,102203.600000,65905.600000


In [82]:
raw_in_match.columns

Index(['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM',
       'INDUSTRY_CD', 'SLS_GRD', 'train', '생활밀접업종', '전체점포수', '프랜차이즈점포수',
       '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률', '폐업률',
       '전체 임대료', '1층 임대료', '1층 외 임대료'],
      dtype='object')

In [83]:
raw_notin_match2.columns

Index(['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'SLS_GRD',
       'train', '전체점포수', '프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수',
       '폐업수', '개업률', '폐업률', '전체 임대료', '1층 임대료', '1층 외 임대료'],
      dtype='object')

In [84]:
# raw_notin_match2에는 행정동코드, 행정동명, 생활밀접업종 열이 없으므로 raw_in_match에서도 drop
raw_in_match.drop(columns=['ADM_CD', 'ADM_NM', '생활밀접업종'], inplace=True)

In [85]:
raw_data1 = pd.concat([raw_in_match, raw_notin_match2]).sort_values(by='DATA_CRTR_YM').reset_index(drop=True)
raw_data1.info() # 초기 raw_data의 length인 1176040과 일치

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176040 entries, 0 to 1176039
Data columns (total 19 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
 6   전체점포수         1176040 non-null  float64
 7   프랜차이즈점포수      1176040 non-null  float64
 8   일반점포수         1176040 non-null  float64
 9   길단위유동인구       1176040 non-null  float64
 10  주거인구          1176040 non-null  float64
 11  직장인구          1176040 non-null  float64
 12  개업수           1176040 non-null  float64
 13  폐업수           1176040 non-null  float64
 14  개업률           1176040 non-null  float64
 15  폐업률           1176040 non-null  float64
 16  전체 임대료        1176040 non-null  float64
 17  1층 임대료        1176040 non-n

In [86]:
raw_data1.isna().sum() # test데이터 10000개이므로 202301,202302 해서 20000개 맞음. 

DATA_CRTR_YM        0
LT_UNQ_NO           0
STDG_EMD_CD         0
INDUSTRY_CD         0
SLS_GRD         20000
train               0
전체점포수               0
프랜차이즈점포수            0
일반점포수               0
길단위유동인구             0
주거인구                0
직장인구                0
개업수                 0
폐업수                 0
개업률                 0
폐업률                 0
전체 임대료              0
1층 임대료              0
1층 외 임대료            0
dtype: int64

In [87]:
raw_data1

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,전체점포수,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,전체 임대료,1층 임대료,1층 외 임대료
0,202201,1111010100100030100,11110101,A03,4.0,1,85.840000,1.450000,84.390000,14929.000000,51.000000,44.000000,3.480000,2.030000,1.189000,0.696000,129755.000000,181103.000000,78407.0
1,202201,1156011100104460000,11560111,A01,5.0,1,295.000000,23.000000,272.000000,92745.000000,278.000000,122.000000,10.000000,7.000000,3.400000,2.400000,93249.000000,115405.000000,71093.0
2,202201,1156011100104450001,11560111,A03,3.0,1,181.830000,25.230000,156.600000,92745.000000,278.000000,122.000000,7.540000,4.350000,1.189000,0.696000,93249.000000,115405.000000,71093.0
3,202201,1156011100104390000,11560111,A04,4.0,1,14.000000,3.000000,11.000000,92745.000000,278.000000,122.000000,0.000000,1.000000,0.000000,7.100000,93249.000000,115405.000000,71093.0
4,202201,1156011100104340000,11560111,A01,5.0,1,295.000000,23.000000,272.000000,92745.000000,278.000000,122.000000,10.000000,7.000000,3.400000,2.400000,93249.000000,115405.000000,71093.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176035,202302,1147010200104060293,11470102,A01,NaN,0,202.000000,37.000000,165.000000,31650.000000,177.000000,111.000000,3.000000,3.000000,1.500000,1.500000,142569.000000,185154.000000,99984.0
1176036,202302,1159010100100950000,11590101,A01,NaN,0,272.000000,23.000000,249.000000,33776.000000,164.000000,31.000000,3.000000,9.000000,1.100000,3.300000,144987.000000,158830.000000,131145.0
1176037,202302,1121510100102110002,11215101,A06,NaN,0,21.000000,5.000000,16.000000,81107.000000,291.000000,25.000000,1.000000,1.000000,4.800000,4.800000,96870.000000,98719.000000,95021.0
1176038,202302,1174010200101940007,11740102,A03,NaN,0,49.300000,13.050000,36.250000,9669.000000,34.000000,8.000000,2.610000,1.450000,1.537000,0.841000,209538.000000,270410.000000,148667.0
